In [4]:
import time
import pandas as pd

In [13]:
# from tqdm import notebook.tqdm as tqdm
from tqdm.notebook import tqdm

In [14]:
# Selenium imports
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [21]:
def init_driver(path):    #path = '../driver/chromedriver.exe'
    # Initialize crome driver
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--profile-directory=Default')
    chrome_options.add_argument('--incognito')
    chrome_options.add_argument('--disable-pligins-discovery')
#     chrome_options.add_argument('--start-maximized')
    
    browser = webdriver.Chrome(executable_path=path, options=chrome_options)
    
    return browser

In [22]:
def search_jobs(job_name:str, job_location:str, num_jobs:int, sleep_time, verbose):
    
    # initialize webdriver
    browser = init_driver(path='../driver/chromedriver.exe') # driver =  browser  path = '../driver/chromedriver.exe'
    url = 'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=data+science&locT=N&locId=96&jobType=&context=Jobs&sc.keyword=data+science&dropdown=0'
    browser.get(url)
#     accept_cookies = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[id='onetrust-accept-btn-handler']"))).click()

    job_title = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Job Title, Keywords, or Company']")))
    location = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Location']")))
    
    job_title.clear()
    location.clear()
    time.sleep(5)  # wait for 5 seconds
    
    job_title.send_keys(job_name)
    location.send_keys(job_location)
    time.sleep(3)
    
    press_submit = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
#     time.sleep(sleep_time)
    time.sleep(3)
    jobs = []
#     tqdm().pandas()
    while len(jobs) < num_jobs:
###################CLOSE SIGN UP POP-UP#############################################        
        time.sleep(sleep_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            browser.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            browser.find_element_by_css_selector('[alt="Close"]').click() #clicking to the X.
            print(' x out worked')
        except NoSuchElementException:
            print(' x out failed')
            pass
####################################################################################        
        # Going through each job in this page
        job_boxes = browser.find_elements_by_class_name("jl")
        for job_box in job_boxes:
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
                
            job_box.click()
            time.sleep(1)
            
            collected_successfully = False
            while not collected_successfully:
                try:
                    company_name = browser.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = browser.find_element_by_xpath('.//div[@class="location"]').text
                    job_nu_naam = browser.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = browser.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text  # The role para on webpage
                    collected_successfully = True
                except:
                    time.sleep(5)
                    
            try:
                salary_estimate = browser.find_element_by_xpath('.//span[@class="css-1uyte9r css-hca4ks e1wijj242"]').text  # css-1uyte9r css-hca4ks e1wijj242
            except NoSuchElementException:
                salary_estimate = -1

            try:
                rating = browser.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1

            if verbose:
                print(f"Job Title: {job_nu_naam}")
                print(f"Salary Estimate: {salart_estimate}")
                print(f"Job Description: {job_description}")
                print(f"Rating: {rating}")
                print(f"Company Name: {company_name}")
                print(f"Location: {location}")

            #Going to the Company tab...
            try:
                browser.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()
                try:
                    headquarters = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text  #//span[@class="value"]
                except NoSuchElementException:
                    size = -1

                try:
                    founded = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

            if verbose:
                print(f"Headquaters: {headquarters}")
                print(f"Size: {size}")
                print(f"Founded: {founded}")
                print(f"Type of Ownership: {type_of_ownership}")
                print(f"Industry: {industry}")
                print(f"Sector:{sector}")
                print(f"Revenue: {revenue}")
                print(f"Competitors: {competitors}")
                print("#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$")

            jobs.append({'Job Title':job_nu_naam,
                        "Salary Estimate" : salary_estimate,
                        "Job Description" : job_description,
                        "Rating" : rating,
                        "Company Name" : company_name,
                        "Location" : location,
                        "Headquarters" : headquarters,
                        "Size" : size,
                        "Founded" : founded,
                        "Type of ownership" : type_of_ownership,
                        "Industry" : industry,
                        "Sector" : sector,
                        "Revenue" : revenue,
                        "Competitors" : competitors})
        try:
            browser.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print(f"Scraping terminated before reaching target of jobs. Needed {num_jobs} , got {len(jobs)}")

    return pd.DataFrame(jobs)


In [23]:
df = search_jobs(job_name='data scientist/data science', job_location='London, England', num_jobs=1000, sleep_time=15, verbose=False)
df

 x out worked
Progress: 0/1000
Progress: 1/1000
Progress: 2/1000
Progress: 3/1000
Progress: 4/1000
Progress: 5/1000
Progress: 6/1000
Progress: 7/1000
Progress: 8/1000
Progress: 9/1000
Progress: 10/1000
Progress: 11/1000
Progress: 12/1000
Progress: 13/1000
Progress: 14/1000
Progress: 15/1000
Progress: 16/1000
Progress: 17/1000
Progress: 18/1000
Progress: 19/1000
Progress: 20/1000
Progress: 21/1000
Progress: 22/1000
Progress: 23/1000
Progress: 24/1000
Progress: 25/1000
Progress: 26/1000
Progress: 27/1000
Progress: 28/1000
Progress: 29/1000
Progress: 30/1000
 x out failed
Progress: 31/1000
Progress: 32/1000
Progress: 33/1000
Progress: 34/1000
Progress: 35/1000
Progress: 36/1000
Progress: 37/1000
Progress: 38/1000
Progress: 39/1000
Progress: 40/1000
Progress: 41/1000
Progress: 42/1000
Progress: 43/1000
Progress: 44/1000
Progress: 45/1000
Progress: 46/1000
Progress: 47/1000
Progress: 48/1000
Progress: 49/1000
Progress: 50/1000
Progress: 51/1000
Progress: 52/1000
Progress: 53/1000
Progress: 

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=87.0.4280.88)


In [48]:
# save file to csv

df.to_csv('raw_data.csv', index=False)